In [ ]:
import os, sys
from datetime import datetime, timedelta,date
import pandas as pd
from IPython.display import display
import xarray as xr
from matplotlib import dates as mdates
import seaborn as sns
import learning_lidar.generation.generation_utils as gen_utils
from scipy.stats import ks_2samp
from learning_lidar.utils import utils, xr_utils, vis_utils, proc_utils, global_settings as gs
vis_utils.set_visualization_settings()
TIMEFORMAT = mdates.DateFormatter(r'%H')
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.mixture import  BayesianGaussianMixture as BayesGMM
from pytictoc import TicToc
import sklearn
import learning_lidar.dataseting.dataseting_utils as ds_utils
from pathlib import Path
import platform
import learning_lidar.generation.valid_utils  as valid_utils
import glob

## 1. Run Settings

In [ ]:
# Data source info
station_name='haifa'
station = gs.Station(station_name)
day_date = datetime(2017,10,2)
daily_times = station.calc_daily_time_index(day_date)
wavelength = 532

# Paths
TGRS_folder = os.path.abspath(os.path.join(gs.PKG_ROOT_DIR ,"..","E","TGRS_GMM_appendix")) if \
    (sys.platform in ['linux', 'ubuntu']) else \
    os.path.abspath(os.path.join(gs.PKG_ROOT_DIR ,"..",'TGRS_GMM_appendix'))
figs_folder = os.path.join(TGRS_folder,'figures_for_talk') # prev: 'figures'
figs_folder = r'C:\Users\addalin\Dropbox\PHD\נסיעות לחול\ICCP pasadena 2022\poster'
model_base_name = 'sklearn_GMM_Model'
models_folder = os.path.join(TGRS_folder,'models')
csv_models = os.path.join(models_folder,'models.csv')
name_str = f"TGRS_TROPOS_{day_date.strftime('%Y-%m-%d')}_profiles"
table_name = name_str+"_info.csv"
profiles_ds_name = name_str+".nc"

# Flags
LOAD_MODEL = True
SAMPLE_BETA = False
GEN_PROFILES = False # True - crate and save the profiles; False - upload previous generated data
SAVE_FIG = True
LOAD_CLIB_CSV = False

# Global print settings
FORMAT_FIG = 'svg' # 'svg'
figsize=(12, 6)  # presentation: (12,5) , manuscript: (8,5)
max_height = 15 # km
height_slice=slice(0,max_height)
sns.set_context('talk')